Name : Chhagani Krunal ,ID : 202001158
Name : Ninama Vipul , ID : 202001218
Name : Harsh metkel , ID : 202001154

In [81]:
import random
from qiskit import QuantumCircuit, Aer, execute,assemble,QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram, plot_bloch_multivector,plot_bloch_vector
from numpy.random import randint
import numpy as np
import qiskit.quantum_info as qi
from qiskit.quantum_info import random_statevector, Statevector
from qiskit import QuantumCircuit, execute
import qiskit.quantum_info as qi
from qiskit.providers.aer import QasmSimulator

In [82]:
# function for applying error to all qubits
def apply_error(qc, qubits, p):
    for qubit in qubits:
        if random.random() < p:
            qc.z(qubit) # flip the phase of this qubit

In [83]:
def apply_hadamard(qc,qubits):
    for qubit in qubits:
        qc.h(qubit)

In [84]:
def phase_flip_circuit():
    # Define the number of qubits
    num_qubits = 1

    # Generate a random statevector

    random_state = random_statevector(2 ** num_qubits)
    # ket 0 state

    zero = Statevector([1,0])

    # Create the circuit
    qr= QuantumRegister(5)
    cr= ClassicalRegister(2)
    qc = QuantumCircuit(qr,cr)  # 5 qubits and 2 classical bits
    qc.initialize(random_state.data,qr[0])
    qc.save_statevector(label='v1')
    initial =  zero ^ zero ^ zero ^ zero ^ random_state  # inital vector is tensor product of 5 qubits

    # Encoding
    qc.cx(0, 1)  # CNOT gate (controlled-X gate) between qubit 0 (main qubit) and qubit 1 (ancillary qubit)
    qc.cx(0, 2)  # CNOT gate between qubit 0 and qubit 2
    qc.save_statevector(label='v2')
    qubits_to_apply_error = [0, 1, 2]  # Apply error to all three qubits in phase and hadamard
    #Apply Hadamard gates to each qubit after encoding since we are working with phase
    apply_hadamard(qc,qubits_to_apply_error)
    qc.save_statevector(label='v3')
    qc.barrier()

    # Apply error
    error_probability = 0.1  # Set the error probability to 0.1
    apply_error(qc, qubits_to_apply_error, error_probability)  # Introduce errors to the qubits
    qc.save_statevector(label='v4')
    qc.barrier()
    #apply Hadamard gates to each qubit after error is introduced
    apply_hadamard(qc,qubits_to_apply_error)
    qc.save_statevector(label='v5')
    qc.barrier()

    # Error Detection
    qc.cx(0, 3)  # C-Z gate between qubit 0 (ancillary qubit) and qubit 3 (error detection qubit)
    qc.cx(1, 3)  # C-Z gate between qubit 1 and qubit 3 (error detection qubit)
    qc.cx(0,4)   # C-Z gate between qubit 0 and qubit 4 (error detection qubit)
    qc.cx(2,4)   # C-Z gate between qubit 1 and qubit 4 (error detection qubit)
    qc.save_statevector(label='v6')
    qc.measure([3,4],[0,1])  # Measure error detection qubits 3 and 4 and store the result in classical bits 0 and 1
    qc.barrier()

    # Apply error correction based on measurement result
    # if measurement gives 00 then simply ignore
    qc.x(1).c_if(cr,1)  # Apply an X gate to qubit 1 if the measurement result is 1 
    qc.x(2).c_if(cr,2) # Apply an X gate to qubit 2 if the measurement result is 2 
    qc.x(0).c_if(cr,3)  # Apply an X gate to qubit 0 if the measurement result is 3
    qc.save_statevector(label='v7')
    qc.barrier()
    # Decoding
    qc.cx(0, 1)  # CNOT gate between qubit 0 and qubit 1
    qc.cx(0,2) # CNOT gate between qubit 0 and qubit 2

    qc.save_statevector(label='v8')
    qc.barrier()
    return qc

In [80]:
#create quantum circuit
qc = phase_flip_circuit()
qc.draw()

┌────────────────────────────────────────────────┐ v1            v2 »
q57_0: ┤ Initialize(-0.26984-0.93583j,0.18501+0.13109j) ├─░────■────■───░──»
       └────────────────────────────────────────────────┘ ░  ┌─┴─┐  │   ░  »
q57_1: ───────────────────────────────────────────────────░──┤ X ├──┼───░──»
                                                          ░  └───┘┌─┴─┐ ░  »
q57_2: ───────────────────────────────────────────────────░───────┤ X ├─░──»
                                                          ░       └───┘ ░  »
q57_3: ───────────────────────────────────────────────────░─────────────░──»
                                                          ░             ░  »
q57_4: ───────────────────────────────────────────────────░─────────────░──»
                                                          ░             ░  »
c11: 2/════════════════════════════════════════════════════════════════════»
                                                                           »
«       ┌───┐ v3  ░  v4  ░ ┌───┐ v5  ░                      v6        ░        »
«q57_0: ┤ H ├─░───░──░───░─┤ H ├─░───░───■─────────■────────░─────────░────────»
«       ├───┤ ░   ░  ░   ░ ├───┤ ░   ░   │         │        ░         ░  ┌───┐ »
«q57_1: ┤ H ├─░───░──░───░─┤ H ├─░───░───┼────■────┼────────░─────────░──┤ X ├─»
«       ├───┤ ░   ░  ░   ░ ├───┤ ░   ░   │    │    │        ░         ░  └─╥─┘ »
«q57_2: ┤ H ├─░───░──░───░─┤ H ├─░───░───┼────┼────┼────■───░─────────░────╫───»
«       └───┘ ░   ░  ░   ░ └───┘ ░   ░ ┌─┴─┐┌─┴─┐  │    │   ░  ┌─┐    ░    ║   »
«q57_3: ──────░───░──░───░───────░───░─┤ X ├┤ X ├──┼────┼───░──┤M├────░────╫───»
«             ░   ░  ░   ░       ░   ░ └───┘└───┘┌─┴─┐┌─┴─┐ ░  └╥┘┌─┐ ░    ║   »
«q57_4: ──────░───░──░───░───────░───░───────────┤ X ├┤ X ├─░───╫─┤M├─░────╫───»
«             ░   ░  ░   ░       ░   ░           └───┘└───┘ ░   ║ └╥┘ ░ ┌──╨──┐»
«c11: 2/════════════════════════════════════════════════════════╩══╩════╡ 0x1 ╞»
«                                                               0  1    └─────┘»
«               ┌───┐  v7  ░            v8  ░ 
«q57_0: ────────┤ X ├──░───░───■────■───░───░─
«               └─╥─┘  ░   ░ ┌─┴─┐  │   ░   ░ 
«q57_1: ──────────╫────░───░─┤ X ├──┼───░───░─
«        ┌───┐    ║    ░   ░ └───┘┌─┴─┐ ░   ░ 
«q57_2: ─┤ X ├────╫────░───░──────┤ X ├─░───░─
«        └─╥─┘    ║    ░   ░      └───┘ ░   ░ 
«q57_3: ───╫──────╫────░───░────────────░───░─
«          ║      ║    ░   ░            ░   ░ 
«q57_4: ───╫──────╫────░───░────────────░───░─
«       ┌──╨──┐┌──╨──┐ ░   ░            ░   ░ 
«c11: 2/╡ 0x2 ╞╡ 0x3 ╞════════════════════════
«       └─────┘└─────┘

In [78]:

# Simulate the circuit
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1)
result = job.result()
counts = result.get_counts(qc)
print(counts)
sv1 = result.data(0)['v1']
sv2 = result.data(0)['v2']
sv3 = result.data(0)['v3']
sv4 = result.data(0)['v4']
sv5 = result.data(0)['v5']
sv6 = result.data(0)['v6']
sv7 = result.data(0)['v7']
sv8 = result.data(0)['v8']


{'10': 1}


In [67]:
sv1.draw('latex', prefix='afterInitializingTheCircuit:')

<IPython.core.display.Latex object>

In [68]:

sv2.draw('latex', prefix='afterEncoding:')


<IPython.core.display.Latex object>

In [69]:

sv3.draw('latex', prefix='afterHadamard:')


<IPython.core.display.Latex object>

In [70]:

sv4.draw('latex', prefix='afterErrorIntroduced:')

<IPython.core.display.Latex object>

In [71]:

sv5.draw('latex', prefix='afterHadamard:')

<IPython.core.display.Latex object>

In [72]:
sv6.draw('latex', prefix='afterErrorDetection:')

<IPython.core.display.Latex object>

In [73]:
sv7.draw('latex', prefix='afterErrorCorrection:')


<IPython.core.display.Latex object>

In [74]:
sv8.draw('latex', prefix='afterDecoding:')

<IPython.core.display.Latex object>

In [75]:
initial = sv1
final = sv8


one = initial.probabilities_dict([0,1,2])
two = final.probabilities_dict([0,1,2])
print(one)
print(two)

{'000': 0.21055285448294234, '001': 0.7894471455170576}
{'000': 0.7894471455170579, '001': 0.21055285448294228}


In [76]:
# sv6.draw('latex', prefix='Statevector6:')
if all(abs(one[qubit] - two[qubit]) < 1e-10 for qubit in one):
    print("Error has been detected and corrected")
else:
    print("Error has not been detected!")

Error has not been detected!
